In [ ]:
#Run if using colab
!pip -q install pybaseball
!pip -q install MLB-StatsAPI


     |████████████████████████████████| 415 kB 4.4 MB/s 
     |████████████████████████████████| 291 kB 46.2 MB/s 
     |████████████████████████████████| 856 kB 37.1 MB/s 
     |████████████████████████████████| 69 kB 2.6 MB/s 


In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0,'') #You may need to include the path to where these folders are stored if using Colab
import BaeBall as bb
import warnings
warnings.filterwarnings("error")
import requests
import os



ModuleNotFoundError: No module named 'NewBaeBall'

## **Generate Training Data**

In [ ]:
outdir='' #choose directory to store feature matrices

#Years used for training
years=list(range(2013,2022))
years.pop(-2) #skipped 2020 for now... COVID
months=list(range(4,9)) #generally season start and end
days=[15,30] #Two dates per month
DFList={}
LogList=[]
ProbList=[]
for year in years:
  for month in months:
    for day in days:
      

      CurrentDate=str(year)+'-'+str(month).zfill(2)+'-'+str(day).zfill(2)
      if os.path.exists(os.path.join(outdir,CurrentDate+'.csv')):
        continue

      feature_matrix,log,progDFs=bb.get_feature_matrix(number_of_batters=200, #see utils file for arguments
                            number_of_games=5,
                            date=CurrentDate,
                            duration_prior=90, 
                            GetMatchupValues=1, #Adjust this as desired
                            train=1,
                            method_few_recent_games='padzero')
      DFList[CurrentDate]=(feature_matrix)

      feature_matrix.to_csv(os.path.join(outdir,CurrentDate+'.csv')) #Save the feature sheet for this day
      LogList.append(log)
      if len(feature_matrix)==0:
        ProbList.append(progDFs)
      print("Finished %s" % CurrentDate)

In [ ]:
#Create one feature matrix of all training data

SheetDir=outdir
FinalDF=pd.DataFrame()
for sheetpath in os.listdir(SheetDir):
  currentsheet=pd.read_csv(os.path.join(SheetDir,sheetpath))
  FinalDF=pd.concat((FinalDF,currentsheet),axis=0)

FinalDF=FinalDF.reset_index(drop=True)
FinalDF.columns
SelectedCols=['IDfg', 'Name', 'Team', 'Age', 'AVG', 'OBP', 'SLG',
       'Contact% (pi)', 'BatterIDs', 'TeamIDs', 'Pitchers', 'Home', 'Ballpark',
       'BallparkNumber', 'PitcherIDs', 'era', 'h9', 'k9', 'whip', 'avg',
       'MatchupAverage', 
       'Game 1', 'Game 2', 'Game 3', 'Game 4', 'Game 5',
       'TestLabels']
FinalizedSheet=FinalDF[SelectedCols]
FinalizedSheet.to_csv('') #Set output name

In [ ]:
#Do the above twice with and without matchup data